# image reconstruction

## Import libraries

In [157]:
import torch
import torchvision
from torch.utils.data import Dataset
from os import listdir
from os.path import join
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from math import log10
from tqdm import tqdm
import os
import random
import copy

## Load data

In [158]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/MachineLearning2022-2/MachineLearning2022-2

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/MachineLearning2022-2/MachineLearning2022-2


In [159]:

directory_data  = './'
filename_data   = 'assignment_11_data.npz'
data            = np.load(os.path.join(directory_data, filename_data))

clean_image_train = data['label_train']
input_image_test  = data['input_test'] 
clean_image_test  = data['label_test']


In [160]:
input_image_test_copy = copy.deepcopy(input_image_test)
clean_image_train_copy = copy.deepcopy(clean_image_train)

input_train 데이터셋 만들기

In [161]:
def idx_list(input_image, idx, temp):
  #temp_input_test에서 0로 채워진 row 찾기
  col_idx_list = []
  row_idx_list = []
  #col
  col_idx_list.append([i for i, value in enumerate(input_image[idx][temp,:]) if value == 0])
  #row
  row_idx_list.append([i for i, value in enumerate(input_image[idx][:,temp]) if value == 0])
  
  idx_list = {'idx':idx, 'row': row_idx_list, 'col': col_idx_list}

  return idx_list

In [162]:
def generate_input_train(clean_image, idx_list, offset):
  #col과 row를 0으로 변환
  idx = idx_list['idx']+offset
  row_idx_list = idx_list['row']
  col_idx_list = idx_list['col']

  input_train_single = copy.deepcopy(clean_image[idx])
  #row
  for i in row_idx_list:
    input_train_single[i] = 0
  #col
  for i in col_idx_list:
    input_train_single[:,i] = 0

  return input_train_single

In [163]:
#input_image_test에서 검은 줄의 row, col
zero_idx = []
for i in range(len(input_image_test_copy)):
  zero_idx.append(idx_list(input_image_test_copy, i,120))

#row, col이 잘못 검출된 이미지의 idx
error_idx=[]
for i in zero_idx:
  row_len = len(i['row'][0])
  col_len = len(i['col'][0])
  if row_len > 50 or col_len > 50 :
    error_idx.append(i['idx'])

error_idx

[]

In [164]:
#0~ 400 : clean_image_train의 zero_idx을 0으로 변환
input_image_train=[]
for i in zero_idx:
  input = generate_input_train(clean_image_train_copy, i,0)
  input_image_train.append(input)

#400~800: clean_image_train의 zero_idx을 0으로 변환
for i in zero_idx:
  input = generate_input_train(clean_image_train_copy, i,400)
  input_image_train.append(input)

#에러 검사
zero_idx_train = []
for i in range(len(input_image_train)):
  zero_idx_train.append(idx_list(input_image_train, i,120))

error_idx_train=[]
for i in zero_idx_train:
  row_len = len(i['row'][0])
  col_len = len(i['col'][0])
  if row_len > 50 or col_len > 50 :
    error_idx_train.append(i['idx'])

error_idx_train

[]

In [165]:
#input_image_train.shape
clean_image_train.shape
input_image_test.shape
clean_image_test.shape

(400, 128, 128)

## plot data

In [166]:
def plot_image(title, image, k):
    
    nRow = 2
    nCol = 2
    size = 3
    
    fig, axes = plt.subplots(nRow, nCol, figsize=(size * nCol, size * nRow))
    fig.suptitle(title, fontsize=16)

    for r in range(nRow):
        for c in range(nCol):
            #k = r * nCol * 10 + c * 4 + 10
            axes[r, c].imshow(image[k], cmap='gray', vmin=0, vmax=1)
            k+=1
    plt.tight_layout()
    plt.show()

In [167]:
#plot_image('input image (testing)', input_image_train, 0)

In [168]:
#plot_image_temp('input image (testing)', input_image_test)

In [169]:
#plot_image('label image (training)', clean_image_train)

In [170]:
#plot_image('clean image (testing)', clean_image_test)

## custom data loader for the training data

In [171]:
class dataset(Dataset):
    
    def __init__(self, clean_image, input_image):
        
        self.clean_image = clean_image
        self.input_image = input_image

    def __getitem__(self, index):
        # ==================================================
        # modify the codes for training data
        #        
        clean_image = self.clean_image[index]
        clean_image = torch.FloatTensor(clean_image).unsqueeze(dim=0)

        input_image = self.input_image[index]
        input_image = torch.FloatTensor(input_image).unsqueeze(dim=0)


        return (clean_image, input_image) # trainingdata도 반환하게 하기, 영상 16:31
        #    
        # ==================================================

    def __len__(self):

        number_image = self.clean_image.shape[0]

        return number_image    

## setting device

In [172]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [173]:
print(device)

cuda


## custom data loader for the testing data

In [174]:
input_image_test = torch.FloatTensor(input_image_test).unsqueeze(dim=1)
clean_image_test = torch.FloatTensor(clean_image_test).unsqueeze(dim=1)

input_image_test = input_image_test.to(device)
clean_image_test = clean_image_test.to(device)

## construct datasets and dataloaders for training and testing

In [175]:
# ==================================================
# determine the mini-batch size
#
size_minibatch      = 10
learning_rate = 0.001
number_epoch=2000
weight_decay    = 0.003
#
# ==================================================

dataset_train       = dataset(clean_image_train, input_image_train)
dataloader_train    = torch.utils.data.DataLoader(dataset_train, batch_size=size_minibatch, shuffle=True, drop_last=True)

## construct a neural network 

In [176]:
class Residual(nn.Module):
    "The Residual block of ResNet."
    def __init__(self, input_channels, num_channels,
                 use_1x1conv=False, strides=1):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, num_channels,
                               kernel_size=3, padding=1, stride=strides)
        self.conv2 = nn.Conv2d(num_channels, num_channels,
                               kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.Conv2d(input_channels, num_channels,
                                   kernel_size=1, stride=strides)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.bn2 = nn.BatchNorm2d(num_channels)

    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        Y += X
        return F.relu(Y)

class Network(nn.Module): 

	def __init__(self, in_channel=1, out_channel=1, dim_feature=8, threshold_ReLU=0.02):
        
		super(Network, self).__init__()

		self.block1 = Residual(in_channel, dim_feature * 1, use_1x1conv=True)
		self.block2 = Residual(dim_feature * 1, dim_feature * 2, use_1x1conv=True)
		self.block3 = Residual(dim_feature * 2, dim_feature * 4, use_1x1conv=True)
		#self.block4 = Residual(dim_feature * 4, dim_feature * 4, use_1x1conv=True)
	
		#self.conv_out = nn.Conv2d(dim_feature * 4, out_channel,	kernel_size=1, stride=1, padding=0, bias=True)
		self.conv_out = nn.Conv2d(dim_feature * 2, out_channel,	kernel_size=1, stride=1, padding=0, bias=True)
		self.activation	= nn.ReLU(inplace=True)
		self.activation_out	= nn.Sigmoid()
		for m in self.modules():
				
				if isinstance(m, nn.Conv2d):

						nn.init.kaiming_uniform_(m.weight) 
						
						if m.bias is not None:

								nn.init.constant_(m.bias, 1)
								pass
								
				elif isinstance(m, nn.BatchNorm2d):
						
						nn.init.constant_(m.weight, 1)
						nn.init.constant_(m.bias, 1)
						
				elif isinstance(m, nn.Linear):
						
						nn.init.kaiming_uniform_(m.weight)

						if m.bias is not None:
								
								nn.init.constant_(m.bias, 1)
								pass




		# *********************************************************************
		# forward propagation
		# *********************************************************************
	def forward(self, x):
		
		x1 = self.block1(x)
		x2 = self.block2(x1)
		#x3 = self.block3(x2)

		y1  = self.conv_out(x2)
		y = self.activation_out(y1)

		return y

In [177]:
"""
#0~1 sigmoid, sigmoid 말고 다른 것 써도 됨
class Network(nn.Module):
    def __init__(self):
        super(Network,self).__init__()

        # -------------------------------------------------
        # Encoder
        # -------------------------------------------------
        self.e_layer1 = nn.Sequential(
                        nn.Conv2d(in_channels=1, out_channels=2, kernel_size=3, stride=1, padding=1, bias=True),  
                        nn.MaxPool2d(2,2),
                        nn.ReLU(),
                        nn.BatchNorm2d(2),
        )
        
        self.e_layer2 = nn.Sequential(
                        nn.Conv2d(in_channels=2, out_channels=4, kernel_size=3, stride=1, padding=1, bias=True),
                        nn.MaxPool2d(2,2),
                        nn.ReLU(),
                        nn.BatchNorm2d(4),
        )

        # -------------------------------------------------
        # Decoder
        # -------------------------------------------------
        self.d_layer1 = nn.Sequential(
                        nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False),
                        nn.Conv2d(in_channels=4, out_channels=2, kernel_size=3, stride=1, padding=1, bias=True),
                        nn.ReLU(),
                        nn.BatchNorm2d(2),
        )
        
        self.d_layer2 = nn.Sequential(
                        nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False),
                        nn.Conv2d(in_channels=2, out_channels=1, kernel_size=3, stride=1, padding=1, bias=True),
                        nn.Sigmoid(),
        )

        # -------------------------------------------------
        # Network
        # -------------------------------------------------
        self.network = nn.Sequential(
                        self.e_layer1,
                        self.e_layer2,
                        self.d_layer1, 
                        self.d_layer2,
        )

        self.initialize_weight()

    def forward(self,x):
    
        out = self.network(x)
      
        return out

    # ======================================================================
    # initialize weights
    # ======================================================================
    def initialize_weight(self):
            
        for m in self.network.modules():
            
            if isinstance(m, nn.Conv2d):

                nn.init.xavier_uniform_(m.weight) 
                if m.bias is not None:

                    nn.init.constant_(m.bias, 1)
                    pass
                    
            elif isinstance(m, nn.BatchNorm2d):
                
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 1)
                
            elif isinstance(m, nn.Linear):

                nn.init.xavier_uniform_(m.weight)

                if m.bias is not None:
                    
                    nn.init.constant_(m.bias, 1)
                    pass
"""

"\n#0~1 sigmoid, sigmoid 말고 다른 것 써도 됨\nclass Network(nn.Module):\n    def __init__(self):\n        super(Network,self).__init__()\n\n        # -------------------------------------------------\n        # Encoder\n        # -------------------------------------------------\n        self.e_layer1 = nn.Sequential(\n                        nn.Conv2d(in_channels=1, out_channels=2, kernel_size=3, stride=1, padding=1, bias=True),  \n                        nn.MaxPool2d(2,2),\n                        nn.ReLU(),\n                        nn.BatchNorm2d(2),\n        )\n        \n        self.e_layer2 = nn.Sequential(\n                        nn.Conv2d(in_channels=2, out_channels=4, kernel_size=3, stride=1, padding=1, bias=True),\n                        nn.MaxPool2d(2,2),\n                        nn.ReLU(),\n                        nn.BatchNorm2d(4),\n        )\n\n        # -------------------------------------------------\n        # Decoder\n        # --------------------------------------------

## build the network

In [178]:
model = Network().to(device)

# ==================================================
# determine the optimiser and its associated hyper-parameters
#

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

#
# ==================================================

## compute the prediction

In [179]:
def compute_prediction(model, input):

    prediction = model(input)

    return prediction

compute the loss

In [180]:
def compute_loss(prediction, label):
    # ==================================================
    # fill up the blank
    #
    criterion   = nn.MSELoss()
    # 
    # ==================================================
    loss        = criterion(prediction, label)

    return loss

compute the accurcy

In [181]:
def compute_accuracy(prediction, label):

    prediction  = prediction.squeeze(axis=1)
    label       = label.squeeze(axis=1)
    mse_loss    = torch.mean((prediction - label) ** 2)

    if mse_loss == 0.0:
        psnr = 100
    else:
        psnr = 10 * torch.log10(1 / mse_loss)

    psnr = psnr.item()
    
    return psnr

## compute the PSNR metric

- data1 : mini-batch-size x channel x height x width (torch tensor)
- data2 : mini-batch-size x channel x height x width (torch tensor)

In [182]:
def compute_psnr(data1, data2):

    mse         = nn.MSELoss()(data1, data2)
    mse_value   = mse.item()
    psnr        = 10 * np.log10(1 / mse_value)

    return psnr

## Variable for the learning curves

In [183]:
psnr_test = np.zeros(number_epoch)

## train

In [184]:
def train(model, optimizer, dataloader):

    model.train()

    # ==================================================
    # fill up the blank
    #
    loss_epoch      = []
    accuracy_epoch  = []

    model.train()

    for index_batch, (original, blur) in enumerate(dataloader):

        original    = original.to(device)
        blur        = blur.to(device)
        
        prediction  = compute_prediction(model, blur)
        loss        = compute_loss(prediction, original)
        accuracy    = compute_accuracy(prediction, original)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_epoch.append(loss.item())
        accuracy_epoch.append(accuracy)

    loss_mean_epoch     = np.mean(loss_epoch)
    loss_std_epoch      = np.std(loss_epoch)

    accuracy_mean_epoch = np.mean(accuracy_epoch)
    accuracy_std_epoch  = np.std(accuracy_epoch)

    loss        = {'mean' : loss_mean_epoch, 'std' : loss_std_epoch}
    accuracy    = {'mean' : accuracy_mean_epoch, 'std' : accuracy_std_epoch}

    return (loss, accuracy)  



    #
    # ==================================================

## test

In [185]:
def test(model, input_image, clean_image):

    model.eval()

    num_steps   = 40
    steps       = np.linspace(0, input_image.shape[0], num_steps+1).astype(int)
    psnr_steps  = np.zeros(num_steps)
    
    for i in range(num_steps):
        input           = input_image[steps[i]:steps[i+1], :, :, :]
        clean           = clean_image[steps[i]:steps[i+1], :, :, :]
        prediction      = compute_prediction(model, input)
        psnr_steps[i]   = compute_psnr(clean, prediction)
    
    psnr = psnr_steps.mean()
    
    return psnr

## train and test

In [ ]:
# ================================================================================
# 
# iterations for epochs
#
# ================================================================================
for i in tqdm(range(number_epoch)):
    
    # ================================================================================
    # 
    # training
    #
    # ================================================================================
    train(model, optimizer, dataloader_train)

    # ================================================================================
    # 
    # testing
    #
    # ================================================================================
    psnr            = test(model, input_image_test, clean_image_test)
    psnr_test[i]    = psnr

 91%|█████████ | 1815/2000 [16:27<01:41,  1.82it/s]

---

## functions for presenting the results

---

In [ ]:
def function_result_01():
    
    title           = 'psnr (testing)'
    label_axis_x    = 'epoch' 
    label_axis_y    = 'psnr'
    
    plt.figure(figsize=(8, 6))
    plt.title(title)

    plt.plot(psnr_test, '-')
    
    plt.xlabel(label_axis_x)
    plt.ylabel(label_axis_y)

    plt.tight_layout()
    plt.show()

In [ ]:
def function_result_02():
    
    nRow = 9
    nCol = 4
    size = 3
    
    title = 'testing results'
    fig, axes = plt.subplots(nRow, nCol, figsize=(size * nCol, size * nRow))
    fig.suptitle(title, fontsize=16)

    model.eval()

    prediction  = compute_prediction(model, input_image_test)

    input_image = input_image_test.detach().cpu().squeeze(axis=1)
    clean_image = clean_image_test.detach().cpu().squeeze(axis=1)
    prediction  = prediction.detach().cpu().squeeze(axis=1)

    nStep = 3

    for r in range(3):
        for c in range(nCol):
            k = r * nCol * 10 + c * 4 + 10
            axes[0 + r * nStep, c].imshow(input_image[k], cmap='gray')
            axes[1 + r * nStep, c].imshow(clean_image[k], cmap='gray', vmin=0, vmax=1)
            axes[2 + r * nStep, c].imshow(prediction[k], cmap='gray', vmin=0, vmax=1)

            axes[0 + r * nStep, c].xaxis.set_visible(False)
            axes[1 + r * nStep, c].xaxis.set_visible(False)
            axes[2 + r * nStep, c].xaxis.set_visible(False)
            
            axes[0 + r * nStep, c].yaxis.set_visible(False)
            axes[1 + r * nStep, c].yaxis.set_visible(False)
            axes[2 + r * nStep, c].yaxis.set_visible(False)

    plt.tight_layout()
    plt.show()

In [ ]:
def function_result_03():
    
    print('final testing psnr = %9.8f' % (psnr_test[-1]))

---

## results 

---

In [ ]:
number_result = 3 

for i in range(number_result):

    title           = '# RESULT # {:02d}'.format(i+1)
    name_function   = 'function_result_{:02d}()'.format(i+1)

    print('') 
    print('################################################################################')
    print('#') 
    print(title)
    print('#') 
    print('################################################################################')
    print('') 

    eval(name_function)

In [ ]:
!apt-get update

In [ ]:
!apt-get install texlive texlive-xetex texlive-latex-extra pandoc
!pip install pypandoc

In [ ]:
!jupyter nbconvert --to PDF '/content/drive/MyDrive/Colab Notebooks/MachineLearning2022-2/MachineLearning2022-2/assignment_11.ipynb'